In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Web Scrapping

- El archivo games que tenemos limpio se compone de varias columnas relevantes sobre cada videojuegos, sin embargo tenemos un problema a la hora de realizar el Content-based filtering (item-item) y es que no tenemos explicitamente lo mas importante, el contenido o algun texto que nos especifique de que trata cada producto. Lo que si poseemos es el link que nos redirije al apartado del videojuego en la pagina oficial de steam cuyo recurso utilizaremos para extraer la sinopsis del juego.

In [59]:
games = pd.read_parquet('./clean_datasets/all_dfs.parquet') # Importamos

In [61]:
games.drop(columns=['posted','user_id','user_url','item_id','recommend','review','sentiment','developer','price'],inplace=True) # Eliminamos las columnas que no necesitamos

In [62]:
print(f'Pre - Filas duplicadas: {games.duplicated().sum()}') ## Validamos si existen algunos registros duplicados
games.drop_duplicates(inplace=True) # Los eliminamos
games.reset_index(drop=True,inplace=True) # Reseteamos los indices
print(f'Pos - Filas duplicadas: {games.duplicated().sum()}') ## Verificamos

Pre - Filas duplicadas: 26839
Pos - Filas duplicadas: 0


In [63]:
games.head() ## Vistazo rapido

,game_title,id,url
0,Killing Floor,1250,http://store.steampowered.com/app/1250/Killing...
1,Zeno Clash,22200,http://store.steampowered.com/app/22200/Zeno_C...
2,Euro Truck Simulator 2,227300,http://store.steampowered.com/app/227300/Euro_...
3,"Papers, Please",239030,http://store.steampowered.com/app/239030/Paper...
4,Risk of Rain,248820,http://store.steampowered.com/app/248820/Risk_...


In [64]:
all = [] ## Creamos la variable donde se almacenaran los fragemntos descriptivos de los videojuegos

for x in games.itertuples():
  page = requests.get(x.url) ## Accedemos a la pagina
  soup = BeautifulSoup(page.content,'html.parser') ## Configuramos la sopa para que pueda leer el contenido de la pagina
  res = soup.find('div',class_='game_description_snippet') ## Accedemos al div con la clase 'game_description_snippet' para obener el resumen del juego

  if res==None: # Para que el largor de la lista concuerde con la cantidad de registros en el DataFrame
    value = "_" 
  else:
    value = res.text.strip()
  
  all.append(value) ## Se agrega el resumen a la lista

games['description'] = pd.Series(all) ## Se añade al DataFrame

In [65]:
games.info() ## Vistazo rapido

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936 entries, 0 to 1935
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   game_title   1936 non-null   object
 1   id           1936 non-null   object
 2   url          1936 non-null   object
 3   description  1936 non-null   object
dtypes: object(4)
memory usage: 60.6+ KB


In [68]:
## Importamos las librerias necesarias

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [69]:
import nltk
from nltk.corpus import stopwords ## Para indicarle al modelo que palabras no poseen ningun significado o relevancia al texto
nltk.download('stopwords')

stop = stopwords.words('english') ## Nuestras descripcion estan en ingles asi que se necesitaran las palabras en ingles

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmoc9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
tf = TfidfVectorizer(stop_words=stop) # Se le indica al modelo que palabras debe ignorar

In [71]:
tfidx_matrix = tf.fit_transform(games['description']) ## Se ajusta el modelo con nuestra columna de descripciones

In [72]:
tf.get_feature_names_out() ## Vistazo breve de terminos que han sido asignadas como columnas para la vectorizacion

array(['00', '000', '000km', ..., '紧接着又是母亲的突然离去', '虽然从未放弃过寻找',
       '让秦幽羽的父亲从此失踪'], dtype=object)

In [74]:
cosine_similarities = linear_kernel(tfidx_matrix,tfidx_matrix) ## Similitud de cosseno entre todos los pares de juegos

In [75]:
n = 5 ## Cantidad de juegos que queremos retornar

results = {} ## Diccionario donde las llaves seran cada juego y los valores los 5 mas parecidos al item

for idx,row in games.iterrows(): ## Se itera el Dataframe
  similar_indexs = cosine_similarities[idx].argsort()[:-n-2:-1] ## Para calcular los indices de los juegos mas similares al juego actual
  
  similar_items = [(f"{games['game_title'][i]}",round(cosine_similarities[idx][i], 3)) for i in similar_indexs] ## Valor de la llave que sera la lista de los juegos similares junto a sus similitudes
  results[f"{row['id']}"] = similar_items[1:] ## Crea la llave con su valor, omitiendo el primero de la lista que seria la similitud del juego consigo mismo (Daria 1.0)
 

In [76]:
game_similarities = pd.DataFrame(pd.Series(results))

In [80]:
game_similarities.to_dict()[0]['10']

[('Team Fortress Classic', 0.195),
 ('Metal Drift', 0.186),
 ('Skara - The Blade Remains', 0.138),
 ('Savage Resurrection', 0.128),
 ('Cyber Team Manager', 0.122)]

In [81]:
game_similarities.to_csv('./clean_datasets/games_similarities.csv')